In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import time
import os

In [2]:
# Headers for request
headers = ({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36', 'Accept-Language':'en-US, en;q=0.5'})

In [3]:
def calculate_discount(original, current):
    try:
        return f"{round((int(original) - int(current)) / int(original) * 100)}%"
    except:
        return ""

In [19]:
existing_ids = set()
products = []
max_pages = 1000

for page in range(1, max_pages + 1):
    print(f"Scraping page {page}...")
    url = f"https://www.daraz.com.bd/catalog/?_keyori=ss&ajax=true&from=search_history&isFirstRequest=true&page={page}&q=mobile"
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"Failed to fetch page {page}")
        break

    data = response.json()
    items = data.get('mods', {}).get('listItems', [])
    if not items:
        print("No items found. Stopping.")
        break

    new_items = 0

    for item in items:
        item_id = str(item.get('itemId'))
        if item_id in existing_ids:
            continue  # Skip duplicates
        existing_ids.add(item_id)
        new_items += 1

        # your existing code to extract data...
        products.append({
            'Item ID': item_id,
            'Title': item.get('name', ''),
            'Price (Raw)': item.get('price', ''),
            'Price (Display)': item.get('priceShow', ''),
            'Original Price': item.get('originalPrice', ''),
            'Original Price (Display)': item.get('originalPriceShow', ''),
            'Discount': calculate_discount(item.get('originalPrice', ''), item.get('price', '')) if item.get('originalPrice') else '',
            'Rating': item.get('ratingScore', ''),
            'Reviews': item.get('review', ''),
            'Seller': item.get('sellerName', ''),
            'Location': item.get('location', ''),
            'Brand': item.get('brandName', ''),
            'Product URL': f"https:{item.get('itemUrl')}" if item.get('itemUrl') else '',
            'Image URL': item.get('image', '')
        })

    print(f"Page {page}: {new_items} new items")

    if new_items == 0:
        print("🛑 No new items found. Reached end of unique products.")
        break

    time.sleep(2)

Scraping page 1...
Page 1: 40 new items
Scraping page 2...
Page 2: 40 new items
Scraping page 3...
Page 3: 40 new items
Scraping page 4...
Page 4: 40 new items
Scraping page 5...
Page 5: 40 new items
Scraping page 6...
Page 6: 40 new items
Scraping page 7...
Page 7: 40 new items
Scraping page 8...
Page 8: 40 new items
Scraping page 9...
Page 9: 40 new items
Scraping page 10...
Page 10: 40 new items
Scraping page 11...
Page 11: 40 new items
Scraping page 12...
Page 12: 40 new items
Scraping page 13...
Page 13: 40 new items
Scraping page 14...
Page 14: 40 new items
Scraping page 15...
Page 15: 40 new items
Scraping page 16...
Page 16: 40 new items
Scraping page 17...
Page 17: 40 new items
Scraping page 18...
Page 18: 40 new items
Scraping page 19...
Page 19: 40 new items
Scraping page 20...
Page 20: 40 new items
Scraping page 21...
Page 21: 40 new items
Scraping page 22...
Page 22: 40 new items
Scraping page 23...
Page 23: 40 new items
Scraping page 24...
Page 24: 40 new items
Scraping p

In [20]:
df = pd.DataFrame(products)

In [21]:
df

,Item ID,Title,Price (Raw),Price (Display),Original Price,Original Price (Display),Discount,Rating,Reviews,Seller,Location,Brand,Product URL,Image URL
0,444402707,iPhone 16 Pro Max (256GB) Global,161499,"৳ 161,499",180000,,10%,5.0,1,SUMASH TECH LIMITED,Dhaka,Apple,https://www.daraz.com.bd/products/iphone-16-pr...,https://static-01.daraz.com.bd/p/41129d667f133...
1,525899327,Motorola Moto G05 (4+64GB),12999,"৳ 12,999",14999,,13%,5.0,3,Phone Palace,Khulna,Motorola,https://www.daraz.com.bd/products/motorola-mot...,https://bd-live-21.slatic.net/kf/S7f907278d498...
2,340092306,Samsung Galaxy S24 Ultra 5G 12GB Ram and 256GB...,109999,"৳ 109,999",129990,,15%,5.0,3,Alapon Computer (CTG),Chattogram,Samsung,https://www.daraz.com.bd/products/samsung-gala...,https://static-01.daraz.com.bd/p/84e96a44fa7d3...
3,438718639,Samsung Galaxy M05 (4+64GB),11999,"৳ 11,999",16999,,29%,5.0,1,Phone Palace,Khulna,Samsung,https://www.daraz.com.bd/products/samsung-gala...,https://bd-live-21.slatic.net/kf/S170b1e8786c9...
4,323700897,iphone 15 Pro Max 256GB/512GB/1TB (USA-LL/A) S...,154999,"৳ 154,999",195990,,21%,5.0,2,Alapon Computer (CTG),Chattogram,,https://www.daraz.com.bd/products/iphone-15-pr...,https://static-01.daraz.com.bd/p/defb4a1f8dd81...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4131,318681133,Xundd Cover For Xiaomi Redmi Note 12 4g // Xun...,110,৳ 110,300,,63%,4.117647058823529,68,RTK Global,Dhaka,No Brand,https://www.daraz.com.bd/products/xundd-cover-...,https://static-01.daraz.com.bd/p/811b5cb8491ef...
4132,320683872,For OnePlus 8 Xundd Shockproof Bumper Case Pho...,120,৳ 120,200,,40%,3.0,2,RTK Classic Gallery,Dhaka,No Brand,https://www.daraz.com.bd/products/oneplus-8-xu...,https://static-01.daraz.com.bd/p/04d10178a3bb5...
4133,221921363,For Oppo product 5D/6D/9D/11D/21D A3s/A5S/A1K/...,57,৳ 57,120,,52%,4.769230769230769,26,m.b telicom,Chattogram,No Brand,https://www.daraz.com.bd/products/oppo-5d6d9d1...,https://static-01.daraz.com.bd/p/a2e00d7eb5b08...
4134,319326601,Rinbo UV Light Adhesive Tempered Glass Screen ...,280,৳ 280,1200,,77%,4.461538461538462,26,COLLECTION GALLERY BD,Dhaka,No Brand,https://www.daraz.com.bd/products/rinbo-uv-lig...,https://static-01.daraz.com.bd/p/bfca337ac7f86...


In [22]:
df.to_csv("C:/Users/mahmu/Downloads/DataAnalyticsProjects/product-recommender-ai/data/daraz_mobile_products.csv", index=False)